In [101]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, QuantileTransformer, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import recall_score, precision_score, precision_recall_curve, f1_score, multilabel_confusion_matrix

model_df = pd.read_csv('../Data/model_data.csv', index_col = 0)
target_encoding = LabelEncoder()
y = model_df['office'].loc[model_df['office'].count() > 1]
y = target_encoding.fit_transform(y)
model_df = model_df.drop(['industry', 'office'], axis = 1)
df_columns = model_df.columns
X = model_df[df_columns]

KeyError: True

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

In [75]:
scaler_standard = StandardScaler()

scaler_quantile = QuantileTransformer()

pca = PCA()

rfc = RandomForestClassifier()

features = FeatureUnion([('standard_scaler', scaler_standard), ('quantile_scaler', scaler_quantile), ('PCA', pca)])

In [76]:
pipeline = Pipeline([('features', features), ('random_forest', rfc)])
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('standard_scaler',
                                                 StandardScaler()),
                                                ('quantile_scaler',
                                                 QuantileTransformer()),
                                                ('PCA', PCA())])),
                ('random_forest', RandomForestClassifier())])

In [77]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('standard_scaler',
                                                 StandardScaler()),
                                                ('quantile_scaler',
                                                 QuantileTransformer()),
                                                ('PCA', PCA())])),
                ('random_forest', RandomForestClassifier())])

In [82]:
y_pred = pipeline.predict(X_test)

f1_score(y_test, y_pred, average = 'weighted')

0.310950397451018

In [92]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy = 'uniform')

dummy.fit(X_train, y_train)

dummy_pred = dummy.predict(X_test)
print(recall_score(y_test, y_pred, average = 'weighted'))
print(precision_score(y_test, y_pred, average = 'weighted', zero_division = 0))
print(f1_score(y_test, dummy_pred, average = 'weighted'))

0.3168141592920354
0.31582673403218964
0.11630798811619157
